 # Batch‐upload up to 10 visitors per event to PolyU’s VRS via the AddRequestv2 API.

In [ ]:
!pip install requests pandas python-dateutil

In [ ]:
import pandas as pd
import requests
from datetime import datetime, timedelta, timezone
from google.colab import files

# 1. Upload files
print("Please upload: Visitor.csv, Event.csv, Cookies.txt")
files.upload()

# 2. Read CSVs
visitors_df = pd.read_csv('Visitor.csv')
events_df   = pd.read_csv('Event.csv')

# 3. Parse only the four cookies into a 'Cookie' header
wanted = {
    'BIGipServerPACI_VISITOR_44301_POOL',
    'BIGipServerWAF_CM_PROD_HTTPS_POOL',
    '.AspNetCore.Session',
    '.AspNetCore.Cookies'
}
cookies = {}
with open('Cookies.txt') as f:
    for line in f:
        if line.startswith('#') or not line.strip(): continue
        parts = line.strip().split('\t')
        if len(parts) >= 7:
            name, val = parts[5], parts[6]
            if name in wanted:
                cookies[name] = val
cookie_header = '; '.join(f"{k}={v}" for k, v in cookies.items())

# 4. Set up session
session = requests.Session()
session.headers.update({
    "Accept": "application/json, text/plain, */*",
    "Content-Type": "application/json",
    "Origin": "https://fmovrs.polyu.edu.hk",
    "Referer": "https://fmovrs.polyu.edu.hk/vrs/requester/request/create",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Cookie": cookie_header
})

API_URL = "https://fmovrs.polyu.edu.hk/vrs-ajax/ShortVisiting/AddRequestv2"

# 5. Date setup
tz = timezone(timedelta(hours=8))
today = datetime.now(tz).date()
max_stay = 13  # days
max_visitors = 10

# 6. Loop through events
for idx, ev in events_df.iterrows():
    # sanitize accessDate
    try:
        ad = datetime.strptime(ev['accessDate'], '%Y-%m-%d').date()
    except:
        ad = today
    if ad < today:
        ad = today

    # sanitize exitDate
    try:
        ed = datetime.strptime(ev['exitDate'], '%Y-%m-%d').date()
    except:
        ed = ad + timedelta(days=max_stay)
    if ed > ad + timedelta(days=max_stay):
        ed = ad + timedelta(days=max_stay)

    ad_str, ed_str = ad.isoformat(), ed.isoformat()

    # prepare the fixed part of the payload
    base_request = {
        "requestId": 0,
        "accessDate":   ad_str,
        "exitDate":     ed_str,
        "purposeOfVisit":    ev['purposeOfVisit'],
        "accessLocation":    ev['accessLocation'],
        "sponsorDepartment": ev['sponsorDepartment'],
        "sponsorName":       ev['sponsorName'],
        "sponsorPhone":      str(ev['sponsorPhone']),
        "sponsorEmail":      ev['sponsorEmail'],
        "sponsorNetID":      ev['sponsorNetID'],
        "visitors":          None,
        "createdAt":   ad_str,
        "createdBy":   ev['sponsorNetID'],
        "updatedAt":   ad_str,
        "updatedBy":   ev['sponsorNetID'],
        "version":     0,
        "status":      "P",
        "approver":    None,
        "remark":      None,
        "visitorCount": None,
        "declarationOption": ""
    }
    disclaimer = {
        "requestId": 0,
        "fromPublicMedia": False,
        "withMediaActivity": False,
        "withApproval": False,
        "healthDeclarationOption": None
    }

    # convert visitors_df to list of dicts for easy slicing
    visitor_dicts = visitors_df.to_dict('records')

    # send in chunks of 10
    for start in range(0, len(visitor_dicts), max_visitors):
        batch = visitor_dicts[start:start + max_visitors]
        payload = {
            "request": base_request.copy(),
            "visitors": [],
            "disclamer": disclaimer
        }

        for v in batch:
            visitor = {
                "visitorId": 0,
                "requestId": 0,
                "lastName":  v['LName'],
                "firstName": v['FName'],
                "mobile":    v.get('Mobile', ""),
                "email":     v['Email'],
                "accesscode": "",
                "organization": ev['organization'],
                "cardNo":       None,
                "visitingStatus": None,
                "checkin":      0,
                "checkout":     0,
                "checkinNotification":  None,
                "checkoutNotification": None,
                "reminderSent": False,
                "updatedAt":    ad_str,
                "updatedBy":    ev['sponsorNetID'],
                "createdAt":    ad_str,
                "createdBy":    ev['sponsorNetID'],
                "version":      0,
                "cardReturned": False
            }
            payload["visitors"].append(visitor)

        resp = session.post(API_URL, json=payload)
        print(f"Event #{idx} batch {start//max_visitors + 1} → HTTP {resp.status_code}", resp.json())
